In [1]:
import torch
import torch.nn as nn
from torch.quantization import quantize_dynamic
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

import performance_test

In [8]:
checkpoint = 'kobart-summarization-finetuned-paper-sample-size-1000/checkpoint-1000'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Dynamic Quantization int 8
GPU에서는 작동을 안하며 생각보다 큰 성능 하락이 보였음.

In [3]:
model_quantized = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)

In [4]:
performance_test.performance_test(model=model_quantized, cpu_flag=True)

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached shuffled indices for dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c/cache-d2fe2f1237e13860.arrow
Loading cached processed dataset at /opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c/cache-538a3728a15b4554.arrow


====ROUGE score====
{'rouge1': 37.0512, 'rouge2': 24.2408, 'rougeL': 33.8331, 'rougeLsum': 33.6528}
Model size (MB) = 211.23576641082764
Average time took(ms) 5.6e+02 +\- 79.79


In [5]:
performance_test.performance_test(model=model, cpu_flag=True)

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

====ROUGE score====
{'rouge1': 40.6664, 'rouge2': 29.6107, 'rougeL': 38.0395, 'rougeLsum': 38.2219}
Model size (MB) = 472.6911554336548
Average time took(ms) 5.2e+02 +\- 67.56


# Half를 사용한 quantization
BatchNorm2d 까지 전부 half() 적용하면 성능 많이 하락함.

In [9]:
model.half()  # convert to half precision
for layer in model.modules():
  if isinstance(layer, nn.BatchNorm2d):
    layer.float()
performance_test.performance_test(model=model, cpu_flag=False)

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c)


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

====ROUGE score====
{'rouge1': 41.3579, 'rouge2': 27.3763, 'rougeL': 37.5986, 'rougeLsum': 37.553}
Model size (MB) = 236.39336490631104
Average time took(ms) 1.2e+02 +\- 1.11
